In [ ]:
from IPython.display import clear_output
import torch
import numpy as np
import pandas as pd
from typing import List

## 모델 진행상황 결과 확인하는 라이브러리
from tqdm.notebook import tqdm

## Huggingface 라이브러리 받아오는 코드
!pip install transformers SentencePiece
!wget https://raw.githubusercontent.com/monologg/KoBERT-Transformers/master/kobert_transformers/tokenization_kobert.py
from transformers import DistilBertModel
from tokenization_kobert import KoBertTokenizer

## 기타 사용할 머신러닝 알고리즘
from sklearn import preprocessing
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

clear_output()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_path = '/content/drive/MyDrive/의현/018_감성대화/Training_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Training.zip'
val_path = '/content/drive/MyDrive/의현/018_감성대화/Validation_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Validation.zip'
!cp -r "$train_path" ./
!cp -r "$val_path" ./
drive.flush_and_unmount()

In [ ]:
!unzip './감성대화말뭉치(최종데이터)_Training.zip'
!unzip './감성대화말뭉치(최종데이터)_Validation.zip'
clear_output()

In [ ]:
import os
import shutil
from sys import platform
from glob import glob

train_data_path = './감성대화말뭉치(최종데이터)_Training.xlsx'
val_data_path = './감성대화말뭉치(최종데이터)_Validation.xlsx'

if platform == "linux" or platform == "linux2":
    pass
elif platform == "darwin":
    train_data_path = os.path.join("dset", train_data_path)
    val_data_path = os.path.join("dset", val_data_path)

In [ ]:
import numpy as np
import pandas as pd

train_dataset = pd.read_excel(train_data_path, index_col = 'Unnamed: 0')
val_dataset = pd.read_excel(val_data_path, index_col = 'Unnamed: 0')

In [ ]:
train_dataset.shape

(51630, 12)

In [ ]:
val_dataset.shape

(6641, 12)

In [ ]:
train_dataset.columns

Index(['연령', '성별', '상황키워드', '신체질환', '감정_대분류', '감정_소분류', '사람문장1', '시스템문장1',
       '사람문장2', '시스템문장2', '사람문장3', '시스템문장3'],
      dtype='object')

In [ ]:
def intro(col):
    uniq = np.unique(train_dataset[col])
    print(f'\n## [{col}]:')
    print(uniq)
    print(f'[{col}] 카테고리 갯수 : {len(uniq)}')

print('unique columns :')
intro('연령')
intro('성별')
intro('상황키워드')
intro('신체질환')
intro('감정_대분류')
intro('감정_소분류')

unique columns :

## [연령]:
['노년' '중년' '청년' '청소년']
[연령] 카테고리 갯수 : 4

## [성별]:
['남성' '여성']
[성별] 카테고리 갯수 : 2

## [상황키워드]:
['가족관계' '건강' '건강,죽음' '대인관계' '대인관계(부부, 자녀)' '연애,결혼,출산' '재정' '재정,은퇴,노후준비'
 '직장, 업무 스트레스' '진로,취업,직장' '학교폭력/따돌림' '학업 및 진로']
[상황키워드] 카테고리 갯수 : 12

## [신체질환]:
['만성질환 무' '만성질환 유' '해당없음']
[신체질환] 카테고리 갯수 : 3

## [감정_대분류]:
['기쁨' '당황' '분노' '불안' '상처' '슬픔']
[감정_대분류] 카테고리 갯수 : 6

## [감정_소분류]:
['가난한, 불우한' '감사하는' '걱정스러운' '고립된' '괴로워하는' '구역질 나는' '기쁨' '낙담한' '남의 시선을 의식하는'
 '노여워하는' '눈물이 나는' '느긋' '당혹스러운' '당황' '두려운' '마비된' '만족스러운' '방어적인' '배신당한'
 '버려진' '부끄러운' '분노' '불안' '비통한' '상처' '성가신' '스트레스 받는' '슬픔' '신뢰하는' '신이 난'
 '실망한' '악의적인' '안달하는' '안도' '억울한' '열등감' '염세적인' '외로운' '우울한' '자신하는' '조심스러운'
 '좌절한' '죄책감의' '질투하는' '짜증내는' '초조한' '충격 받은' '취약한' '툴툴대는' '편안한' '한심한' '혐오스러운'
 '혼란스러운' '환멸을 느끼는' '회의적인' '후회되는' '흥분' '희생된']
[감정_소분류] 카테고리 갯수 : 58


In [ ]:
train_dataset.groupby(['감정_대분류', '감정_소분류']).agg({'신체질환':'count'})

신체질환
감정_대분류 감정_소분류           
기쁨     감사하는          631
       기쁨            660
       느긋            568
       만족스러운         655
       신뢰하는          609
       신이 난          634
       안도            639
       자신하는          570
       편안한           597
       흥분            563
당황     고립된           900
       남의 시선을 의식하는   872
       당황            859
       부끄러운          897
       열등감           859
       외로운           865
       죄책감의          883
       한심한           918
       혐오스러운         833
       혼란스러운         870
분노     구역질 나는        882
       노여워하는         975
       방어적인          803
       분노            958
       성가신           969
       악의적인          845
       안달하는          873
       좌절한           875
       짜증내는         1054
       툴툴대는          926
불안     걱정스러운        1229
       당혹스러운         842
       두려운           991
       불안            893
       스트레스 받는       971
       조심스러운         869
       초조한           900
       취약한           864
       혼란스러운         901
       회의적인          860
상처     가난한, 불우한      949
       고립된           902
       괴로워하는         953
       배신당한          911
       버려진           861
       상처            937
       억울한           984
       질투하는          896
       충격 받은         910
       희생된           840
슬픔     낙담한           896
       눈물이 나는        993
       마비된           838
       비통한           950
       슬픔            958
       실망한           870
       염세적인          828
       우울한          1002
       환멸을 느끼는       884
       후회되는          906

In [ ]:
train_dataset[['사람문장1', '사람문장2', '사람문장3']].describe()

,사람문장1,사람문장2,사람문장3
count,51630,51630,42695
unique,51601,50832,42295
top,요즘 너무 힘들어.,대인관계가 원만하지 않아.,눈물만 흘렸어. 내가 할 수 있는 건 아무것도 없어.
freq,5,4,8


In [ ]:
train_dataset[['사람문장1', '사람문장2', '사람문장3']] = train_dataset[['사람문장1', '사람문장2', '사람문장3']].fillna('').astype(str)
train_dataset['sentence'] = train_dataset[['사람문장1', '사람문장2', '사람문장3']].apply(lambda x: ' '.join(x), axis=1)
train_dataset = train_dataset.drop(['사람문장1', '사람문장2', '사람문장3', '시스템문장1', '시스템문장2', '시스템문장3'], axis=1)

In [ ]:
train_dataset['sentiment'] = train_dataset[['감정_대분류', '감정_소분류']].apply(lambda x: ' '.join(x), axis=1)
train_dataset = train_dataset.drop(['감정_대분류', '감정_소분류'], axis=1)

In [ ]:
train_dataset.head()

,연령,성별,상황키워드,신체질환,sentence,sentiment
1,청년,여성,"진로,취업,직장",해당없음,일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한...,분노 노여워하는
2,청년,여성,"진로,취업,직장",해당없음,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....,분노 노여워하는
3,청년,여성,"진로,취업,직장",해당없음,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,분노 노여워하는
4,청년,여성,"진로,취업,직장",해당없음,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,분노 노여워하는
5,청년,여성,"진로,취업,직장",해당없음,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나. 상사인 나에게 ...,분노 노여워하는


In [ ]:
val_dataset[['사람문장1', '사람문장2', '사람문장3']] = val_dataset[['사람문장1', '사람문장2', '사람문장3']].fillna('').astype(str)
val_dataset['sentence'] = val_dataset[['사람문장1', '사람문장2', '사람문장3']].apply(lambda x: ' '.join(x), axis=1)
val_dataset = val_dataset.drop(['사람문장1', '사람문장2', '사람문장3', '시스템문장1', '시스템문장2', '시스템문장3'], axis=1)
val_dataset['sentiment'] = val_dataset[['감정_대분류', '감정_소분류']].apply(lambda x: ' '.join(x), axis=1)
val_dataset = val_dataset.drop(['감정_대분류', '감정_소분류'], axis=1)

In [ ]:
X_train = train_dataset.drop('sentiment', axis = 1)
y_train = train_dataset['sentiment']
X_val = val_dataset.drop('sentiment', axis = 1)
y_val = val_dataset['sentiment']

In [ ]:
class classification_with_koBERT:
    def __init__(self):
        self.tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
        self.model = DistilBertModel.from_pretrained('monologg/distilkobert').to(device)

    def extract(self, texts: List[str]):
        batch_size = 16
        features = np.zeros((len(texts), 768), dtype = np.float16)

        for i in tqdm(range(0, len(texts), batch_size)):
            batch_texts = texts[i:i+batch_size]
            tokenized_text = self.tokenizer(batch_texts, return_tensors="pt", max_length=512, truncation=True, padding=True).to(device)
            model_output = self.model(**tokenized_text)[0].detach().cpu()
            features[i:i+batch_size, :] = model_output.numpy().mean(axis=1)

        return features

In [ ]:
le = preprocessing.LabelEncoder()

train_labels = le.fit_transform(y_train.values)

In [ ]:
## BERT, standardscaler, XGBoost 정의
extractor = classification_with_koBERT()
scaler = StandardScaler()
classifier = XGBClassifier(use_label_encoder = False)

## XGBoost 학습
texts = [", ".join(str(_)) for _ in X_train.values]
train_features = scaler.fit_transform(extractor.extract(texts))
classifier.fit(train_features, train_labels)

## Prediction with Test Data
answer = y_val.values
texts = [", ".join(str(_)) for _ in X_val.values]
preds = classifier.predict(scaler.transform(extractor.extract(texts)))


## 모델 예측의 정밀도 측정!
accuracy = accuracy_score(answer, le.inverse_transform(preds))
print(f'\naccuracy : {accuracy*100:.2f}%')

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


  0%|          | 0/3227 [00:00<?, ?it/s]

  0%|          | 0/416 [00:00<?, ?it/s]


accuracy : 26.02%


In [ ]:
import pickle

filename = 'text_classification(xgboost).sav'
pickle.dump(classifier, open(filename, 'wb'))

loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
txt = '무려 1003일만에 한화이글스 4연승!'
  x``
loaded_model.predict(scaler.transform(extractor.extract(txt)))

  0%|          | 0/416 [00:00<?, ?it/s]

array([32, 13, 24, ..., 34, 32, 41])